# Projeto de PO-233 
Este projeto busca realizar análise do problema proposto pelo Data Science Challenge 2019 - ITA.

##### Alunos:
- Fernando Zanchitta
- Davi Xie
- Hugo Timóteo

##### Dados
Os dados podem ser adquiridos no site: https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques/data

## Introdução

O campo do aprendizado de máquina e da ciência de dados tem se mostrado cada vez mais relevante e aplicável em diversas áreas. Entre as áreas de grande interesse encontra-se a precificação de imóveis residenciais, um desafio que envolve a análise de várias características do imóvel e a estimativa do seu valor de mercado. Essa é uma tarefa complexa, devido à grande quantidade de fatores que influenciam o problema. Características como tamanho, localização, qualidade, idade, entre outras, desempenham um papel fundamental na determinação do preço de venda. No entanto, a relação entre essas características e o preço não é linear e muitas vezes não é facilmente mensurável. Além disso, existem fatores externos e tendências de mercado que também impactam o valor dos imóveis. Portanto, é um desafio desenvolver modelos que sejam capazes de estimar com precisão os preços de venda.

Com base nisso, objetivo deste trabalho é explorar o uso de técnicas de aprendizado indutivo, análise exploratória de dados e aprendizado preditivo para construir um modelo de regressão capaz de estimar o preço de venda de imóveis residenciais. Para isso, utilizaremos o conjunto de dados "House Prices - Advanced Regression Techniques", que contém informações detalhadas sobre características dos imóveis, como tamanho, localização, qualidade, idade, entre outras.

A fim de se alcançar o objetivo proposto, inicialmente será realizda uma análise exploratória dos dados, aplicando estatísticas descritivas e visualização multivariada para compreender a distribuição e relação entre as variáveis do conjunto de dados. Em seguida, faremos o pré-processamento dos dados, incluindo limpeza, redução dimensional e transformações, a fim de preparar os dados para a construção do modelo de regressão. Em sequência, serão utilizadas técnicas de aprendizado preditivo para treinar e avaliar diferentes modelos de regressão, utilizando métricas de desempenho adequadas para a tarefa de precificação de imóveis. Por fim, serão avaliadas a capacidade de generalização dos modelos por meio de validação cruzada e análise das métricas de erro.



## Descrição da base de dados



## Importando as bibliotecas necessárias

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from sklearn.impute import KNNImputer
import functions
from sklearn.feature_selection import mutual_info_regression
from scipy.stats import chi2_contingency

In [ ]:
import os
print(os.getcwd())

## Realizando a leitura do dataframe

In [ ]:
train = pd.read_csv('../dataset/train.csv', index_col = 'Id')

In [ ]:
train.head()


## Pré-Processamento

### Remover Dados Faltantes:
Existem técnicas de inserção de variáveis que irão ser mostradas posteriormente. Entretanto, para alguns tipos de atributos cujo percentual de variáveis faltando é muito alto, fazer o tratamento dos valores faltantes gera muito ruído para predição. Portanto, decidiu-se retirar os atributos com percentual de valores faltantes maior que $50\%$

In [ ]:
# check missing values:

missing_values = pd.DataFrame(data={
    'Feature_name': train.columns,
    'missing_values': train.isnull().sum(),
    'percentage': train.isnull().sum() / len(train) * 100,
    'type': train.dtypes
})
missing_values.sort_values(by='percentage', ascending=False).head(20)

In [ ]:
#remover colunas com mais de 50% de valores faltantes:
features_to_drop = missing_values[missing_values['percentage'] > 50]['Feature_name'].values
train = train.drop(features_to_drop, axis='columns')

In [ ]:
train

### Atributos redundantes
Nessa seção, investigaremos se há algum atributo com um único valor. Esses atributos não possuem poder preditivo sobre o modelo, e poderiam ser descartados.

In [ ]:
# contagem de valores únicos:
unique_values = pd.DataFrame(data={"Feature_name": train.columns, "unique_values": train.nunique()})

In [ ]:
unique_values.sort_values(by='unique_values', ascending=True).head(20)

Não temos nenhum atributo com valores únicos que possa ser descartado a priori

## Análise Exploratória

A análise exploratória de dados é um processo inicial na análise de dados, no qual o objetivo é obter uma compreensão básica dos dados e identificar padrões e tendências. É uma etapa crucial antes de aplicar técnicas estatísticas mais avançadas ou construir modelos preditivos.

### Análise Exploratória para Dados Quantitativos:

In [ ]:
# Determina os atributos numéricos
numerical_features = train.select_dtypes(include=['int64', 'float64']).columns

# Determina os atributos quantitativos discretos e continuos


discreet_quantitative = []
continuous_quantitative = []
for column in train[numerical_features].columns:
    unique_values = train[numerical_features][column].nunique()
    if unique_values <= 16:
        discreet_quantitative.append(column)
    else:
        continuous_quantitative.append(column)    

# Exibe as colunas selecionadas
print(discreet_quantitative)
print(continuous_quantitative)

In [ ]:
print("quantidade de atributos quantitativos discretos: ", len(discreet_quantitative))
print("quantidade de atributos quantitativos continuos: ", len(continuous_quantitative))

#### Resumo estatístico: 

Calculam-se medidas estatísticas como média, mediana, moda, desvio padrão, mínimo, máximo e quartis. Essas medidas fornecem uma visão geral das características centrais, dispersão e distribuição dos dados.

In [ ]:
# Calcula o resumo estatístico usando a função describe()
summary = train[continuous_quantitative].describe().transpose()
    
# Adiciona a moda como uma nova coluna no resumo estatístico
summary['moda'] = train[continuous_quantitative].mode().transpose()[0]
    
# Define os nomes das colunas da tabela
colunas = ['Atributo', 'Média', 'Mediana', 'Moda', 'Desvio Padrão', 'Mínimo', '25%', '50%', '75%', 'Máximo']
    
# Cria a matriz com o resumo estatístico
matriz_resumo = pd.DataFrame(columns=colunas)
matriz_resumo['Atributo'] = summary.index
matriz_resumo[['Média', 'Mediana', 'Moda', 'Desvio Padrão', 'Mínimo', '25%', '50%', '75%', 'Máximo']] = summary[['mean', '50%', 'moda', 'std', 'min', '25%', '50%', '75%', 'max']].values

# Imprime a tabela
display(matriz_resumo)    

#### Visualização de dados:
Utilizam-se gráficos e visualizações adequados para representar os dados quantitativos. Isso pode incluir histogramas, box plots, gráficos de dispersão, gráficos de linha ou gráficos de séries temporais, dependendo da natureza dos dados e do objetivo da análise.
- Gráficos de dispersão

In [ ]:
# Configurar o estilo do Seaborn
sns.set()

# Definir o número de subplots por linha e o tamanho da figura
subplots_per_row = 3
figsize = (15, 15)

# Obter todas as colunas de atributos do dataframe train (exceto a última coluna)
attribute_columns = train[continuous_quantitative].columns[:-1]

# Calcular o número total de subplots
num_subplots = len(attribute_columns)

# Calcular o número de linhas
num_rows = (num_subplots - 1) // subplots_per_row + 1

# Criar a figura e os subplots
fig, axes = plt.subplots(num_rows, subplots_per_row, figsize=figsize)

# Converter a matriz de subplots em um array unidimensional
axes = axes.flatten()

# Iterar sobre os atributos e criar os gráficos de dispersão
for i, attribute in enumerate(attribute_columns):
    # Selecionar o subplot atual
    ax = axes[i]
    
    # Criar o gráfico de dispersão
    sns.scatterplot(x=attribute, y=train.columns[-1], data=train, ax=ax)
    
    # Definir o título do gráfico
    ax.set_title(f'{attribute} vs SalePrice')
    
    # Definir o nome do eixo y como o nome do atributo alvo
    ax.set_ylabel(train[continuous_quantitative].columns[-1])
    
# Remover os subplots vazios, se houverem
if num_subplots < len(axes):
    for j in range(num_subplots, len(axes)):
        fig.delaxes(axes[j])

# Ajustar o espaçamento entre os subplots
fig.tight_layout()

# Exibir o gráfico
plt.show()

- Histogramas

In [ ]:
# Configurar o estilo do Seaborn
sns.set()

# Definir o número de subplots por linha e o tamanho da figura
subplots_per_row = 3
figsize = (15, 15)

# Obter todas as colunas de atributos do dataframe train (exceto a última coluna)
attribute_columns = train[continuous_quantitative].columns[:-1]

# Calcular o número total de subplots
num_subplots = len(attribute_columns)

# Calcular o número de linhas
num_rows = (num_subplots - 1) // subplots_per_row + 1

# Criar a figura e os subplots
fig, axes = plt.subplots(num_rows, subplots_per_row, figsize=figsize)

# Converter a matriz de subplots em um array unidimensional
axes = axes.flatten()

# Iterar sobre os atributos e criar os histogramas com as curvas de distribuição
for i, attribute in enumerate(attribute_columns):
    # Selecionar o subplot atual
    ax = axes[i]
    
    # Criar o histograma com a curva de distribuição
    sns.histplot(data=train, x=attribute, kde=True, ax=ax)
    
    # Definir o título do subplot
    ax.set_title(attribute)
    
# Remover os subplots vazios, se houverem
if num_subplots < len(axes):
    for j in range(num_subplots, len(axes)):
        fig.delaxes(axes[j])

# Ajustar o espaçamento entre os subplots
fig.tight_layout()

# Exibir o gráfico
plt.show()

#### Análise de correlação:
Explora-se a relação entre diferentes variáveis quantitativas por meio de medidas de correlação, como o coeficiente de correlação de Pearson. Isso ajuda a identificar a força e a direção da relação entre as variáveis.

In [ ]:
corr_matrix = train[continuous_quantitative].corr()
plt.subplots(figsize=(12,9))
plt.title('Matriz de correlação entre as variáveis numéricas.')
sns.heatmap(corr_matrix, vmax=0.9, square=True)

Vamos ver quais variáveis tem correlação alta com o nosso alvo: _SalesPrice_:

In [ ]:
# correlação entre as variáveis numéricas e o preço:
corr_matrix['SalePrice'].sort_values(ascending=False).head(12)

Intuitivamente falando, é provavel que as variáveis mais correlacionadas tenham maior poder preditivo sobre o preço. Entretando devemos verificar se há correlação entre elas mesmas, para fins de simplificação, vamos pegar os atributos cuja correlação seja maior que $0.5$

In [ ]:
#atributos com correlação maior que 0.5:
features_correlated = corr_matrix['SalePrice'].sort_values(ascending=False).loc[lambda x : x > 0.5].index

#plotar a correlação entre as  variáveis com correlação maior que 0.5:
corr_matrix = train[features_correlated].corr()
plt.subplots(figsize=(12,9))
plt.title('Matriz de correlação entre as variáveis numéricas com correlação maior que 0.5.')
sns.heatmap(corr_matrix, vmax=0.9, square=True)


Observamos que algumas variáveis são fortemente correlacionadas: 
-  _GarageArea_ e _GarageCars_ : Faz sentido se pensar que o aumento do numero de carros na garagem exige uma garagem maior.

- _TotalBsmtSF_ e _1stFlrSF_: A relação entre o total de metragem do imovel com a metragem do primeiro andar também é pertinente.

- _TotRmsAbvGrd_ e _GrLivArea_: O número total de salas em relação ao tamanho da sala de estar também é pertinente


Portanto vamos deletar _1stFlrSF_, _GarageArea_ e _GrLivArea_ (Escolhidos de forma arbitrária).

In [ ]:
# remover as variáveis com correlação maior que 0.5:
features_correlated = features_correlated.drop(['GarageArea', '1stFlrSF', 'GrLivArea'])
train = train.drop(['GarageArea', '1stFlrSF', 'GrLivArea'], axis='columns')
# test = test.drop(['GarageArea', '1stFlrSF', 'GrLivArea'], axis='columns')

In [ ]:
# Determina novamente os atributos numéricos após a exclusão de dados
numerical_features = train.select_dtypes(include=['int64', 'float64']).columns

# Determina os atributos quantitativos discretos e continuos

discreet_quantitative = []
continuous_quantitative = []
for column in train[numerical_features].columns:
    unique_values = train[numerical_features][column].nunique()
    if unique_values <= 16:
        discreet_quantitative.append(column)
    else:
        continuous_quantitative.append(column)    

# Exibe as colunas selecionadas
print(discreet_quantitative)
print("Quantidade de atributos discretos: ", len(discreet_quantitative))
print(continuous_quantitative)
print("Quantidade de atributos contínuos: ", len(continuous_quantitative))

#### Identificação de outliers: 

Vamos verificar se há valores outliers nos atributos e vamos realizar o tratamento deles. Inicialmente iremos fazer uma inspeção visual por meio do plot de boxplots

In [ ]:
# Configurar o estilo do Seaborn
sns.set()

# Configurar o layout dos subplots
n = 4 # Número de subplots por linha
num_features = len(continuous_quantitative)
num_subplots = num_features // n + (num_features % n > 0)
fig, axes = plt.subplots(num_subplots, 4, figsize=(15, num_subplots * 5))

# Iterar sobre as features numéricas (exceto a coluna do atributo alvo)
for i, feature in enumerate(continuous_quantitative):
    ax = axes[i // n, i % n]  # Acessar o subplot correspondente
    sns.boxplot(y=train[feature], ax=ax, orient='v')  # Plotar boxplot na vertical
    ax.set_title(feature)  # Definir o título do subplot

# Remover subplots vazios, se necessário
if num_features % n != 0:
    for i in range(num_features % n, n):
        fig.delaxes(axes[-1, i])

# Ajustar o espaçamento entre os subplots
fig.tight_layout()

# Exibir o gráfico
plt.show()

##### Identificando Outlier com Z- Score
O z-score nos da uma idéia do quanto um determinado ponto está afastado da média dos dados, isto é , ele mede quantos desvios padrão abaixo ou acima da média populacional ou amostral os dados estão:
$$
z=\frac{x-\mu}{\sigma}
$$
Onde:
- x: observação
- $\mu$: média
- $\sigma$: desvio padrão
Assumindo uma distribuição normal, sabe-se que 99,7% dos dados estão à uma distância de três desvios padrão da média. Com base nisso, será considerado nesse trabalho que dados com distância acima de três desvios padrão serão considerados outliers.

In [ ]:
# # Filtrar apenas as colunas numéricas do DataFrame train após o drop
# numerical_features = train.select_dtypes(include=['int64', 'float64']).columns

# # Criar uma cópia do DataFrame train
# train_ZS = train[continuous_quantitative]

# # Calcular e atribuir o Z-score para cada coluna numérica, exceto a última
# for col in continuous_quantitative[:-1]:
#     col_values = train_ZS[col].values
#     zscore = stats.zscore(col_values)  # Calcula o Z-score para todos os valores da coluna
#     outliers = (zscore > 3) | (zscore < -3)
#     train_ZS.loc[outliers, col] = np.nan

# # Criação do imputador KNN
# imputer = KNNImputer(n_neighbors=15, weights='uniform', metric='nan_euclidean')

# # Ajuste do imputador aos dados
# imputer.fit(train_ZS)

# # Imputação dos valores ausentes
# train_ZS = pd.DataFrame(imputer.transform(train_ZS), columns=train_ZS.columns)

##### Identificando Outlier com Amplitude interquartil

Percentil:

- percentil 25 : primeiro quaril
- percentil 50 : segundo quartil ou mediana
- percentil 75 : terceito quartil

Amplitude interquartil:

É Diferença entre o terceiro quartil (Q3) e o primeiro quartil (Q1)
Para identificar Outlier rom amplitude interquartil serão realizados os procedimentos abaixo:
1. Ordenar os dados de forma crescente;
2. Calcular o valor do primeiro e terceiro quartil
3. Determinar a amplitude interquartil
4. Encontrar o limite inferior =  Q1 - 1,5*amplitude interquartil
5. Encontrar o limite superior =  Q3 + 1,5*amplitude interquartil

In [ ]:
# # Criar novo DataFrame com as colunas numéricas
# train_IQR = train[continuous_quantitative]

# # Calcular o IQR e identificar outliers
# for col in continuous_quantitative[:-1]:
#     Q1 = train_IQR[col].quantile(0.25)
#     Q3 = train_IQR[col].quantile(0.75)
#     IQR = Q3 - Q1
    
#     limIn = Q1 - (IQR * 1.5)
#     limSp = Q3 + (IQR * 1.5)
    
#     # Substituir outliers por np.nan
#     train_IQR.loc[(train_IQR[col] < limIn) | (train_IQR[col] > limSp), col] = np.nan

# # Criar imputador KNN
# imputer = KNNImputer(n_neighbors=15, weights='uniform', metric='nan_euclidean')

# # Ajustar imputador aos dados
# imputer.fit(train_IQR)

# # Imputar valores ausentes
# train_IQR = pd.DataFrame(imputer.transform(train_IQR), columns=train_IQR.columns)

#### Imputando valores aos outliers
Em ambos os métodos acima, foram identificados os outliers e para cada um deles foi atribuido um valor ausente, ou seja, eles foram excluidos dos dados.
Para preencher os dados ausentes foi utilizada uma técnica de imputação de valores com base em proximidade, denominada KNN. O imputador KNN (K-Nearest Neighbors) é uma técnica dque pode ser utilizada para preencher valores ausentes em conjuntos de dados. Ele é um algoritmo de aprendizado de máquina capaz de prever valores ausentes com base na similaridade entre as amostras do conjunto de dados.

Inicialmente ele encontra os K vizinhos mais próximos (específicamente 15 nesse trabalho) para cada valor ausente, calculando a distância entre a amostra com valor ausente e todas as outras amostras no conjunto de dados. Ele seleciona as K amostras mais próximas com base em uma métrica de distância que, neste caso, é a distância euclidiana.

Em seguida, o KNN calcula um valor imputado para o valor ausente com base na média arimética dos valores dos vizinhos (vizinhos com pesos iguais). Por fim, o valor imputado é atribuído a cada valor ausente no conjunto de dados.

#### Análise Exploratória para Dados Qualitativos:
Inicialmente vamos converter os atributos do tipo 'object' para 'category'

In [ ]:
# Selecionar as colunas do tipo 'object'
object_columns = train.select_dtypes(include='object').columns

# Converter as colunas para o tipo 'category'
train[object_columns] = train[object_columns].astype('category')

Após isso, vamos selecionar os dados qualitativos

In [ ]:
# categorical = train.columns[~train.columns.isin(train[continuous_quantitative].columns)]

In [ ]:
# numerical: discrete
discrete = [
    var for var in train.columns if train[var].dtype != 'category' and var != 'SalePrice'
    and train[var].nunique() < 10
]

# numerical: continuous
continuous = [
    var for var in train.columns
    if train[var].dtype != 'category' and var != 'SalePrice' and var not in discrete
]

# categorical
categorical = [var for var in train.columns if train[var].dtype == 'category' and train[var].nunique() > 1]

print('There are {} discrete variables'.format(len(discrete)))
print('There are {} continuous variables'.format(len(continuous)))
print('There are {} categorical variables'.format(len(categorical)))
# print("numerical_features: ",len(numerical_features))
# print("category_features: ",len(category_features))



Para dados qualitativos, que envolvem atributos ou categorias, a análise exploratória pode envolver os seguintes aspectos:

#### Frequência e distribuição: 
Calculam-se as frequências absolutas e relativas de cada categoria presente nos dados qualitativos. Isso permite entender a distribuição e proporções de cada categoria.

In [ ]:
# Dividindo os atributos em duas metades
half = len(train[categorical]) // 2
first_half = train[categorical][:half]
second_half = train[categorical][half:]

# Configurações do estilo seaborn
sns.set(style='whitegrid')

# Configuração dos subplots para a primeira metade dos atributos
num_first_half = len(first_half.columns)
num_rows_first_half = (num_first_half - 1) // 4 + 1
fig, axes = plt.subplots(nrows=num_rows_first_half, ncols=4, figsize=(15, 4*num_rows_first_half))

# Plotagem dos histogramas para a primeira metade dos atributos
for i, col in enumerate(first_half.columns):
    ax = axes[i // 4, i % 4]
    sns.histplot(data=train, x=col, ax=ax, stat='count', discrete=True)
    ax.set_title(col, fontsize=10)
    ax.set_xlabel('')
    ax.set_ylabel('')

    # Definir as etiquetas do eixo x na vertical
    ax.set_xticklabels(ax.get_xticklabels(), rotation='vertical')
    

# Remover subplots vazios, se houverem
if num_first_half < num_rows_first_half * 4:
    for i in range(num_first_half, num_rows_first_half * 4):
        fig.delaxes(axes[i // 4, i % 4])

# Ajuste de espaçamento entre subplots
plt.tight_layout()

# Configuração dos subplots para a segunda metade dos atributos
num_second_half = len(second_half.columns)
num_rows_second_half = (num_second_half - 1) // 4 + 1
fig, axes = plt.subplots(nrows=num_rows_second_half, ncols=4, figsize=(15, 4*num_rows_second_half))

# Plotagem dos histogramas para a segunda metade dos atributos
for i, col in enumerate(second_half.columns):
    ax = axes[i // 4, i % 4]
    sns.histplot(data=train, x=col, ax=ax, stat='count', discrete=True)
    ax.set_title(col, fontsize=10)
    ax.set_xlabel('')
    ax.set_ylabel('')

    # Definir as etiquetas do eixo x na vertical
    ax.set_xticklabels(ax.get_xticklabels(), rotation='vertical')
    

# Remover subplots vazios, se houverem
if num_second_half < num_rows_second_half * 4:
    for i in range(num_second_half, num_rows_second_half * 4):
        fig.delaxes(axes[i // 4, i % 4])

# Ajuste de espaçamento entre subplots
plt.tight_layout()

# Exibição dos plots
plt.show()

#### Distribuição do atributo alvo em relação aos demais atributos

A distribuição do atributo alvo em relação aos demais atributos refere-se à relação entre a variável alvo (também conhecida como variável dependente, resposta ou variável a ser prevista) e as outras variáveis do conjunto de dados (também conhecidas como variáveis independentes, preditoras ou explicativas).

Essa análise busca compreender como a distribuição ou os valores da variável alvo variam ou são influenciados pelas diferentes categorias ou níveis das outras variáveis. Em outras palavras, examina-se como a variável alvo se comporta em relação a diferentes valores ou grupos das outras variáveis.

In [ ]:
# Converter atributos qualitativos para tipo categórico
# for col in categorical:
#     train[col] = train[col].astype('category')

    # Verificar valores ausentes
    # if train[col].isnull().any():
        # train[col] = train[col].cat.add_categories(['MISSING'])
        # train[col] = train[col].fillna('MISSING')

def boxplot(x, y, **kwargs):
    sns.boxplot(x=x, y=y)
    plt.xticks(rotation=90)

# Derreter o DataFrame
melted = pd.melt(train, id_vars=['SalePrice'], value_vars=categorical)

# Configurar o FacetGrid para os boxplots
g = sns.FacetGrid(melted, col="variable", col_wrap=4, sharex=False, sharey=False, height=5)

# Mapear a função boxplot no FacetGrid
g.map(boxplot, "value", "SalePrice")

# Exibir os plots
plt.show()


#### Análise de variância (ANOVA) 

A ANOVA é um teste estatístico utilizado para comparar a média de um grupo com a média de outros grupos, permitindo determinar se existem diferenças significativas entre os grupos. No gráfico do ANOVA, a disparidade (disparity) é representada no eixo y em uma escala logarítmica. Valores mais altos indicam uma maior disparidade entre as categorias da variável qualitativa em relação à variável de resposta (SalePrice no caso desse código).

Em termos práticos, uma disparidade alta indica que as categorias da variável qualitativa têm uma influência significativa na variável de resposta e podem ser consideradas importantes para explicar as variações nos valores da variável de resposta. Isso sugere que a variável qualitativa pode ser um bom preditor da variável de resposta.

Por outro lado, uma disparidade baixa indica que as categorias da variável qualitativa têm pouca influência ou não são estatisticamente significativas na explicação das variações na variável de resposta. Isso sugere que a variável qualitativa pode não ser relevante ou informativa para prever a variável de resposta.

In [ ]:
anv = pd.DataFrame()
anv['feature'] = train[categorical].columns
pvals = []
for c in train[categorical].columns:
    samples = []
    for cls in train[train[categorical][c].notnull()][c].unique():
        s = train[train[categorical][c] == cls]['SalePrice'].values
        samples.append(s)
    pval = stats.f_oneway(*samples)[1]
    pvals.append(pval)
anv['pval'] = pvals
anv['disparity'] = np.log(1. / np.maximum(anv['pval'].values, 1e-100))
anv = anv.sort_values('disparity', ascending=False)
plt.figure(figsize=(15, 6))
sns.barplot(data=anv, x='feature', y='disparity')
plt.xticks(rotation=90)
plt.xlabel('Feature')
plt.ylabel('Disparity (log-scale)')
plt.title('ANOVA - Disparity of Qualitative Features')
plt.show()



#### Teste qui-quadrado:
O teste qui-quadrado, também conhecido como teste de qui-quadrado de Pearson, é um teste estatístico utilizado para determinar se existe uma associação significativa entre duas variáveis categóricas. Ele baseia-se na comparação entre as frequências observadas e as frequências esperadas sob a hipótese nula de que não há associação entre as variáveis. A ideia é verificar se as diferenças observadas entre as frequências são grandes o suficiente para serem consideradas estatisticamente significativas.

O valor de p-value é uma medida estatística que indica a evidência contra a hipótese nula em um teste estatístico. Em geral, ele é usado para avaliar se existe uma associação significativa entre duas variáveis em um teste de independência, como o teste do qui-quadrado.

- Um p-value baixo (geralmente menor que 0,05 ou 0,01) indica que há evidências estatísticas significativas para rejeitar a hipótese nula. Isso sugere que existe uma associação ou diferença estatisticamente significativa entre as variáveis testadas.

- Um p-value alto (geralmente maior que 0,05 ou 0,01) indica que não há evidências suficientes para rejeitar a hipótese nula. Isso sugere que não há uma associação ou diferença estatisticamente significativa entre as variáveis testadas.

In [ ]:
# Realizar o teste qui-quadrado e obter os resultados
results = []
for column in train[categorical].columns:
    contingency_table = pd.crosstab(train[column], train['SalePrice'])
    chi2, p_value, _, _ = chi2_contingency(contingency_table)
    results.append((column, chi2, p_value))

# Criar um DataFrame com os resultados
results_df = pd.DataFrame(results, columns=['Feature', 'Chi2', 'P-value'])

# Ordenar os resultados pelo valor de p-value (do menor para o maior)
results_df.sort_values(by='P-value', inplace=True)

# Plotar o gráfico de barras dos valores de p-value
plt.figure(figsize=(12, 6))
sns.barplot(data=results_df, x='Feature', y='P-value')
plt.xticks(rotation=90)
plt.xlabel('Feature')
plt.ylabel('P-value')
plt.title('Teste Qui-quadrado: Valores de P-value por Feature')
plt.tight_layout()
plt.show()

as variáveis para o modelo são as seguintes:

In [ ]:
features_correlated

# Engenharia de Atributos
Nessa seção vamos realizar insersão de valores, e codificação de atributos

In [ ]:
train.dtypes

In [ ]:
from sklearn.model_selection import train_test_split

X = train.drop(['SalePrice'], axis=1).copy()
Y = train['SalePrice'] # here we need to remove unnacessary columns if exist

train_X, test_X, train_Y, test_Y = train_test_split(X, Y, test_size=0.3, random_state=1)

## Valores faltantes
Vamos verificar as variáveis com valores faltantes. Para as variáveis com uma porcentagem menor que 5%

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [ ]:
# check if is null mean greater than 0
train_X.isnull().mean().sort_values(ascending=False).head(20)

Vamos deletar a variável _FireplaceQu_ e as demais, realizaremos um processo de inserção de dados. Para as variáveis contínuas, iremos realizar uma imputação simples pela mediana. Já para as variáveis discretas e categóricas iremos pela frequência. 

In [ ]:
continuous_imputer = SimpleImputer(strategy='mean')
discrete_imputer = SimpleImputer(strategy='most_frequent')
categorical_imputer = SimpleImputer(strategy='most_frequent')

continuous_imputer.fit(train_X[continuous])
discrete_imputer.fit(train_X[discrete]) 
categorical_imputer.fit(train_X[categorical])

Agora o _SimpleImputer_ aprendeu a partir dos dados de treinamento, e realizará a imputação dos valores faltantes em todo o conjunto de dados de treinamento e teste. Nessa etapa não há vazamento de informações do teste para o treino.

In [ ]:
train_X[continuous] = continuous_imputer.transform(train_X[continuous])
train_X[discrete] = discrete_imputer.transform(train_X[discrete])
train_X[categorical] = categorical_imputer.transform(train_X[categorical])

test_X[continuous] = continuous_imputer.transform(test_X[continuous])
test_X[discrete] = discrete_imputer.transform(test_X[discrete])
test_X[categorical] = categorical_imputer.transform(test_X[categorical])


Verificamos que a insersção de dados foi feita corretamente.

In [ ]:

train_X.isnull().mean().sort_values(ascending=False).head()

In [ ]:
test_X.isnull().mean().sort_values(ascending=False).head()

Não há valores faltantes.

## Rare Labels
É comum encontrarmos em grandes datasets alguns atrbutos que contêm categorias raras. Para aumentar o poder preditivo dos modelos e diminuir o número de categorias, emprega-se o **RareLabel Encoding**.

In [ ]:
from feature_engine import encoding as ce
rare_label_encoder = ce.RareLabelEncoder(
    tol=0.05, # porcentagem mínima de observações para não ser considerado raro
    n_categories=1,# numero de categorias para considerar raro
    )
rare_label_encoder.fit(train_X[categorical+discrete])


In [ ]:
train_X[categorical+discrete] = rare_label_encoder.transform(train_X[categorical+discrete])
test_X[categorical+discrete] = rare_label_encoder.transform(test_X[categorical+discrete])

Para verificar o resultado, basta procurarmos a categoria "Rare" no conjunto de dados:

In [ ]:
#procurar por celulas com valore "Rare":
train_X[categorical+discrete].apply(lambda x: x.str.contains('Rare').any(), axis=1).sum()

## Conversão de variáveis categóricas
Existem diversas técnicas de conversão simbólica-numérica, dentre elas as mais comuns são: codificação binária, ordinal, por frequência, e pela mediana.

Neste projeto, a opção escolhida foi a codificação ordinal, uma vez que muitas variáveis categóricas preservam algum certo de grau de progressão ou intensidade.

In [ ]:
ordinal_encoder = ce.OrdinalEncoder(encoding_method='ordered',variables=categorical)
ordinal_encoder.fit(train_X,train_Y)


In [ ]:
train_X = ordinal_encoder.transform(train_X)
test_X = ordinal_encoder.transform(test_X)
train_X[categorical].head()

In [ ]:
train_X

## Outliers
Vamos verificar se há valores outliers nos atributos e vamos realizar o tratamento deles
### Identificando Outlier com Z- Score
O z-score nos da uma idéia do quanto um determinado ponto está afastado da média dos dados, isto é , ele mede quantos desvios padrão abaixo ou acima da média populacional ou amostral os dados estão:
$$
z=\frac{x-\mu}{\sigma}
$$
Onde:
- x: observação
- $\mu$: média
- $\sigma$: desvio padrão
Assumindo uma distribuição normal, sabe-se que 99,7% dos dados estão à uma distância de três desvios padrão da média. Com base nisso, será considerado nesse trabalho que dados com distância acima de três desvios padrão serão considerados outliers.

In [ ]:
continuous

In [ ]:
# # Filtrar apenas as colunas numéricas do DataFrame train após o drop
# # numerical_features = train.select_dtypes(include=['int64', 'float64']).columns

# # Criar uma cópia do DataFrame train
# train_ZS = train[continuous]

# # Calcular e atribuir o Z-score para cada coluna numérica, exceto a última
# for col in numerical_features[:-1]:
#     col_values = train_ZS[col].values
#     zscore = stats.zscore(col_values)  # Calcula o Z-score para todos os valores da coluna
#     outliers = (zscore > 3) | (zscore < -3)
#     train_ZS.loc[outliers, col] = np.nan

# # Criação do imputador KNN
# imputer = KNNImputer(n_neighbors=15, weights='uniform', metric='nan_euclidean')

# # Ajuste do imputador aos dados
# imputer.fit(train_ZS)

# # Imputação dos valores ausentes
# train_ZS = pd.DataFrame(imputer.transform(train_ZS), columns=train_ZS.columns)

# caminho_arquivo_excel = '../dataset/train_ZS.xlsx'
# train_ZS.to_excel(caminho_arquivo_excel, index=False)

### Identificando Outlier com Amplitude interquartil
#### Percentil
- percentil 25 : primeiro quaril
- percentil 50 : segundo quartil ou mediana
- percentil 75 : terceito quartil
#### Amplitude interquartil
É Diferença entre o terceiro quartil (Q3) e o primeiro quartil (Q1)
Para identificar Outlier rom amplitude interquartil serão realizados os procedimentos abaixo:
1. Ordenar os dados de forma crescente;
2. Calcular o valor do primeiro e terceiro quartil
3. Determinar a amplitude interquartil
4. Encontrar o limite inferior =  Q1 - 1,5*amplitude interquartil
5. Encontrar o limite superior =  Q3 + 1,5*amplitude interquartil

In [ ]:
# from feature_engine.outliers import OutlierTrimmer
# # ver com Analise Exploratoria de Dados os dados que estão fora do padrão
# trimmer = OutlierTrimmer(
#     variables=continuous,
#     capping_method="iqr",
#     tail="both",
#     fold=1.5,
# )

# trimmer.fit(train_X)

In [ ]:
# Criar novo DataFrame com as colunas numéricas
# Calcular o IQR e identificar outliers
# for col in continuous:
#     Q1 = train_X[col].quantile(0.25)
#     Q3 = train_X[col].quantile(0.75)
#     IQR = Q3 - Q1
    
#     limIn = Q1 - (IQR * 1.5)
#     limSp = Q3 + (IQR * 1.5)
    
#     # Substituir outliers por np.nan
#     # train_IQR.loc[(x[col] < limIn) | (train_IQR[col] > limSp), col] = np.nan
#     inliers = train_X[col].between(limIn, limSp)
#     train_X = train_X.loc[inliers]

#     inliers = test_X[col].between(limIn, limSp)
#     test_X = test_X.loc[inliers]

# Criar imputador KNN
# imputer = KNNImputer(n_neighbors=15, weights='uniform', metric='nan_euclidean')

# Ajustar imputador aos dados
# imputer.fit(train_IQR)

# Imputar valores ausentes
# train_IQR = pd.DataFrame(imputer.transform(train_IQR), columns=train_IQR.columns)

# caminho_arquivo_excel = '../dataset/train_IQR.xlsx'
# train_IQR.to_excel(caminho_arquivo_excel, index=False)

# caminho_arquivo_excel = '../dataset/train_2.xlsx'
# train[numerical_features].to_excel(caminho_arquivo_excel, index=False)

In [ ]:
train_X

#### Imputando valores aos outliers
Em ambos os métodos acima, foram identificados os outliers e para cada um deles foi atribuido um valor ausente, ou seja, eles foram excluidos dos dados.
Para preencher os dados ausentes foi utilizada uma técnica de imputação de valores com base em proximidade, denominada KNN. O imputador KNN (K-Nearest Neighbors) é uma técnica dque pode ser utilizada para preencher valores ausentes em conjuntos de dados. Ele é um algoritmo de aprendizado de máquina capaz de prever valores ausentes com base na similaridade entre as amostras do conjunto de dados.

Inicialmente ele encontra os K vizinhos mais próximos (específicamente 15 nesse trabalho) para cada valor ausente, calculando a distância entre a amostra com valor ausente e todas as outras amostras no conjunto de dados. Ele seleciona as K amostras mais próximas com base em uma métrica de distância que, neste caso, é a distância euclidiana.

Em seguida, o KNN calcula um valor imputado para o valor ausente com base na média arimética dos valores dos vizinhos (vizinhos com pesos iguais). Por fim, o valor imputado é atribuído a cada valor ausente no conjunto de dados.

# Seleção de Atributos (opcional)
Nessa seção vamos realizar a seleção de atributos relevantes para o modelo.


# Treino e Teste

Nessa seção vamos treinar um modelo de regressão que gere uma predição dos valores de teste

In [ ]:
# regressao linear
from sklearn.linear_model import LinearRegression

#arvore de deciao
from sklearn.tree import DecisionTreeRegressor

# SVM
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.svm import SVR

# Adaline
from sklearn.linear_model import SGDRegressor

#MLP
from sklearn.neural_network import MLPRegressor

# Naive Bayes
from sklearn.naive_bayes import MultinomialNB, GaussianNB

#cross validation
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate


Separamos o dataset de treino na proporção de 70-30 em um dataset de treino e outro de teste. Perceba que o tamanho do dataset de teste realmente é 30% do dataset original.

In [ ]:
print(train_X.shape, train_Y.shape)
print(test_X.shape, len(test_Y))

### Regressão Linear

O primeiro modelo será uma regressão linear utilizando a biblioteca do scikit-learn de linear_model importando a classe LinearRegression

In [ ]:
LR = LinearRegression()
LR.fit(train_X, train_Y)

LR_predicted = LR.predict(test_X)

### Árvore de Decisão

O segundo modelo será um regressor de Árvore de Decisão utilizando a biblioteca do scikit-learn de tree importando a classe DecisionTreeRegressor

In [ ]:
DTR = DecisionTreeRegressor(criterion='squared_error', max_depth=15, min_samples_split=5, min_samples_leaf=5)
DTR.fit(train_X, train_Y)

DTR_predicted = DTR.predict(test_X)

### SVM

O terceiro modelo será um SVM utilizando a biblioteca do scikit-learn de svm importando a classe SVR

In [ ]:
SVM = make_pipeline(RobustScaler(), SVR(kernel='linear', C=1000))
SVM.fit(train_X, train_Y)

SVM_predicted = SVM.predict(test_X)

### Adaline - Redes Neurais

In [ ]:
ADALINE = SGDRegressor(loss='huber', learning_rate='constant', eta0=0.01, max_iter=1000)
ADALINE.fit(train_X, train_Y)

ADALINE_predicted = ADALINE.predict(test_X)

### MLP - Redes Neurais

In [ ]:
MLP = MLPRegressor(hidden_layer_sizes=(64, 32), activation='relu', solver='adam', max_iter=10000, random_state=42)

MLP.fit(train_X, train_Y)
MLP_predicted = MLP.predict(test_X)

### Naive Bayes

In [ ]:
NB = GaussianNB(var_smoothing=1e-9)

NB.fit(train_X, train_Y)
NB_predicted = NB.predict(test_X)

# Avaliação do Modelo

Nessa seção vamos avaliar a performance do modelo gerado.

Os principais critério que vamos avaliar vão ser os seguintes:
- $R^2$:
O primeiro é o coeficiente de determinação, usualmente expresso por $R^2$. O coeficiente de determinação é a razão da variância do alvo, explicado ou predito pelo modelo, pela a variância total do alvo. É um valor com limites entre $0$ e $1$, e quanto mais próximo de 1 maior a capacidade do modelo em explicar ou prever a variância da variável alvo.
$$R^2 = \frac{\sum(\hat{y} - \bar{y})^2}{\sum(y - \bar{y})^2}$$

- $RMSE$:
A segunda métrica é o erro médio quadrático (_Mean Squared Error_). O $MSE$ é a média das diferenças entre o valor alvo predito e o valor real ao quadrado. Nesse sentido, é sempre maior que zero, e quanto menor o valor de $MSE$ maior a acurácia das predições do modelo. Nesse projeto, iremos utilizar a raiz quadrada de $MSE$, chamada de $RMSE$.
$$RMSE = \sqrt{\frac{1}{n}\sum^n_{i=1}(y_i-p_i)^2}$$

In [ ]:
# avaliando
from sklearn.metrics import mean_squared_error, r2_score
from functions.actual_vs_pred_plot import actual_vs_pred_plot
from functions.model_residual_plot import model_residual_plot
from functions.regression_metrics import regression_metrics
from functions.model_dist_plot import model_dist_plot
import seaborn as sns

list_models_name = [
    "Regressão Linear",
    "Árvore de Decisão",
    "SVM",
    "ADALINE",
    "MLP",
    "Naive Bayes",
]
list_models = [LR, DTR, SVM, ADALINE, MLP, NB]
list_models_predicted = [LR_predicted, DTR_predicted, SVM_predicted, ADALINE_predicted, MLP_predicted, NB_predicted]
list_models_RMSE = []
list_models_R2 = []
list_models_MAPE = []

for i in range(0, len(list_models)):
    model_predicted = list_models_predicted[i]
    model_name = list_models_name[i]
    print("\nModelo " + model_name + ":")
    rmse, r2, mape = regression_metrics(model_predicted, test_Y,model_name)
    list_models_RMSE.append(rmse)
    list_models_R2.append(r2)
    list_models_MAPE.append(mape)

    model_dist_plot(test_Y, model_predicted, model_name)
    
    actual_vs_pred_plot(test_Y, model_predicted,model_name)

    # Create residual plot
    model_residual_plot(test_Y, model_predicted, model_name)


Comparando os valores de $MAPE$ (erro percentual absoluto médio) de cada modelo gerado, obtemos:

In [ ]:
# comparando os modelos por MAPE:
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 5))
ax = sns.barplot(x=list_models_name, y=list_models_MAPE)

É possível perceber que SVM é o modelo que possui a menor média dos erros percentuais absolutos entre os valores preditos e os valores verdadeiros, enquanto que os modelos de Redes Neurais possuíram altas médias.

Comparando os $RMSE$ de cada modelo gerado.

In [ ]:
# comparando os modelos por RMSE:
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 5))
ax = sns.barplot(x=list_models_name, y=list_models_RMSE)

Da mesma forma, é possível perceber que Árvore de Decisão é o modelo com o menor RMSE, enquanto que novamente os modelos de redes neurais Adaline e MLP tiveram valores altos de RMSE.

Comparando os scores de $R2$ de cada modelo gerado, obtemos:

In [ ]:
# comparando os modelos por R2:
sns.set_theme(style="whitegrid")
plt.figure(figsize=(10, 5))
ax = sns.barplot(x=list_models_name, y=list_models_R2)

Novamente, Árvore de Decisão se destaca como o melhor modelo nesse quesito, onde quanto mais próximo de 1 é o score de R_2, melhor é o ajuste do modelo aos dados.

$ADALINE$ e $MLP$ são ambos modelos de redes neurais, sendo que a $MLP$ é mais complexa do que o $ADALINE$. Redes neurais exigem um grande volume de dados para aprender adequadamente, e podem ter desempenho abaixo do esperado se houver poucos dados de treinamento. Além disso, esses modelos podem sofrer de overfitting se não forem adequadamente configurados. Isso poderia explicar os resultados elevados de MAPE e RMSE (indicativos de erro) e baixos valores de R2 (indicativo de acurácia).

Por outro lado, o $SVM$, a $Regressão$ $Linear$ e a $Árvore$ $de$ $Decisão$ são algoritmos mais simples e menos propensos ao overfitting. O $SVM$ é especialmente conhecido por sua habilidade em lidar com espaços de alta dimensionalidade e encontrar fronteiras de decisão complexas, o que pode explicar por que ele tem o menor MAPE. No entanto, isso não significa necessariamente que ele terá o menor RMSE ou o maior R2, já que essas métricas avaliam aspectos ligeiramente diferentes do modelo. O fato de a $Árvore$ $de$ $Decisão$ ter o menor RMSE e o maior R2 pode indicar que ela está fazendo um bom trabalho de generalização.

O $Naive$ $Bayes$ faz uma suposição de independência entre os atributos que raramente é verdadeira na prática, o que pode limitar sua acurácia. Se os atributos em seus dados são altamente correlacionados (como muitas vezes acontece em dados de imóveis), isso poderia explicar por que o $Naive$ $Bayes$ não tem desempenho tão bom quanto os outros modelos, mas ainda assim é melhor do que o $ADALINE$ e o $MLP$.

Portanto, seguindo a métrica de _Root Mean Squared Error_ como a principal para fazer seleção dos modelos, o melhor modelo foi:

In [ ]:
model_index = np.argmin(list_models_RMSE)
model_name = list_models_name[model_index]
model_predicted = list_models_predicted[model_index]
print("\nModelo com melhor R2: " + model_name)
print("MAPE: " + str(list_models_MAPE[model_index]))
print("RMSE: " + str(list_models_RMSE[model_index]))
print("R2: " + str(list_models_R2[model_index]))

In [ ]:
# from sklearn.impute import SimpleImputer

# melhor_modelo = list_models[model_index]

# validate_columns = features_correlated.drop(['SalePrice'])
# validate_df = test[validate_columns]

# # Eliminate these lines after - Preprocess the data to handle missing values
# # imputer = SimpleImputer(strategy='mean')
# # validate_df = pd.DataFrame(imputer.fit_transform(validate_df), columns=validate_columns)

# validate = validate_df.copy()
# validate['SalePrice'] = melhor_modelo.predict(validate_df)

# # validate

# predicted_sale_price = validate
# predicted_sale_price.to_csv("SalePrice_predicted.csv", index=False)
# predicted_sale_price.head()

# Implementando Pipelines e Validação cruzada

# Considerações finais